<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>
<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto tem o intuito de studar Modelos de Séries Temporais para previsão de informações sobre o Covid19 no Brasil. </i>       
</p>  

# <font color='red' style='font-size: 40px;'> Problemática </font>
<hr style='border: 2px solid red;'>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i>  Este projeto tem o intuito de studar Modelos de Séries Temporais para previsão de informações sobre o Covid19 no Brasil.


</i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> 
</i> 
</p>  


https://www.kaggle.com/datasets/unanimad/corona-virus-brazil?select=brazil_covid19_cities.csv

https://www.kaggle.com/code/mauriciofigueiredo/an-lise-e-previs-o-de-s-ries-temporais-covid


# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [1]:
## Bibliotecas De Manipulação de Dados e Visualização
import pandas as pd 
import builtins as builtins
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import SparkSession, Row, functions as F 
import matplotlib.pyplot as plt
import seaborn as sns 
from IPython.display import display, Image

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import scipy as sp 
import scipy.stats as stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import normaltest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, kruskal
from statsmodels.stats.weightstats import ztest
from statsmodels.stats.diagnostic import lilliefors

# Bibliotecas de Manipulação de Tempo
import time
import datetime

# Bibiliotecas de Seleção de Modelos
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer 
from sklearn.feature_selection import RFE, VarianceThreshold
from feature_engine.selection import DropConstantFeatures, DropCorrelatedFeatures, SmartCorrelatedSelection

# Bibliotecas de Pré-Processamento e Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder, BinaryEncoder
import category_encoders as ce 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer

# Bibliotecas de Modelos de Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Bibliotecas de Métricas de Machine Learning
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Parâmetros de Otimização
import warnings
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Spark Session
spark = SparkSession.builder.getOrCreate()

# <font color='red' style='font-size: 40px;'> Funções </font>
<hr style='border: 2px solid red;'>

In [2]:
def plota_temporal(df, titulo, x, y):
    plt.figure(figsize=(10, 6))
    plt.plot(df, color='#1FB3E5', linewidth=2)
    plt.scatter(df.index, df, color='#1FB3E5', s=40)
    plt.title(f"{titulo}", fontsize=14)  # Define o título do gráfico e o tamanho da fonte
    plt.xlabel(f"{x}", fontsize=12)  # Define o rótulo do eixo x e o tamanho da fonte
    plt.ylabel(f"{y}", fontsize=12)  # Define o rótulo do eixo y e o tamanho da fonte
    plt.xticks(df.index, fontsize=10)  # Define o tamanho da fonte dos valores no eixo x
    plt.yticks(fontsize=10)  # Define o tamanho da fonte dos valores no eixo y
    plt.grid(True, linestyle=':', alpha=0.5)  # Adiciona linhas de grade
    plt.tight_layout()  # Ajusta automaticamente a posição dos elementos do gráfico
    plt.show()  # Exibe o gráfico

In [3]:
def ano_mes_dia(df):

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day


In [4]:
def separa_feature_target(target, dados):
    x = dados.drop(target, axis = 1)
    y = dados[[target]]
    return x, y

In [5]:
def separa_treino_teste_temporal(data, data_teste, dados):
    dados.sort_values(by = data, ascending = True, inplace = True)
    df_train = dados.loc[dados[data] < data_teste]
    df_test = dados.loc[dados[data] >= data_teste]

    return df_train, df_test

In [6]:
def discretiza_variavel(df, variavel_quant, variavel_qualit, bins, labels, right):
    df[variavel_qualit] = pd.cut(
        df[variavel_quant], 
        bins= bins, 
        labels= labels, 
        right = right
    )
    df.drop(variavel_quant, axis = 1, inplace = True)

In [7]:
def metricas(regressor, target, y_train, y_predict_train, y_test, y_predict_test, coeficiente_det_train, coeficiente_det_test):
    y_test[target].fillna(y_train[target].median(), inplace = True)

    mae_train = mean_absolute_error(y_predict_train, y_train)
    mse_train = mean_squared_error(y_predict_train, y_train)
    mape_train = mean_absolute_percentage_error(y_predict_train, y_train)
    metricas_treino = pd.DataFrame({'Coeficiente de Determinação (R^2)':coeficiente_det_train, 'Erro_Medio_Absoluto':mae_train, 'Erro_Quadratico_Medio':mse_train, 'Media_Percentual_Absoluta_Erro':mape_train, 'Etapa':'treino', 'Regressor':regressor}, index = np.arange(1, 2))


    mae_test = mean_absolute_error(y_predict_test, y_test)
    mse_test = mean_squared_error(y_predict_test, y_test)
    mape_test = mean_absolute_percentage_error(y_predict_test, y_test)
    metricas_teste = pd.DataFrame({'Coeficiente de Determinação (R^2)':coeficiente_det_test, 'Erro_Medio_Absoluto':mae_test, 'Erro_Quadratico_Medio':mse_test, 'Media_Percentual_Absoluta_Erro':mape_test, 'Etapa':'teste', 'Regressor':regressor}, index = np.arange(1, 2))

    metricas_finais = pd.concat([metricas_treino, metricas_teste])

    return metricas_finais

def metricas_modelos_juntos(lista_modelos):
    metricas_modelos = pd.concat(lista_modelos).set_index('Regressor')
    return metricas_modelos


# <font color='red' style='font-size: 40px;'> 1) Leitura do Dataset </font>
<hr style='border: 2px solid red;'>

# <font color='green' style='font-size: 30px;'> 1.1) Descrição das Variáveis </font>
<hr style='border: 2px solid green;'>

## 1.1.1) Cases

- **Date:** Anomesdia.
- **State:** Estado.
- **Name:** Nome do Estado.
- **Code:** Código.
- **Cases:** Casos.
- **Deaths:** Mortes.

## 1.2) Cities

- **State Code:** Código do Estado.
- **City Code:** Código da Cidade.
- **City Name:** Nome da Cidade.
- **Lat:** Latitude.
- **Long:** Longitude.
- **Capital:** Flag para saber se é uma capital.

## 1.3) Population

- **Region:** Região.
- **State:** Nome do Estado.
- **City:** Nome da Cidade.
- **City Code:** Código da Cidade.
- **Health Region Code:** Código da Região de Saúde.
- **Health Region:** Nome da Região de Saúde.
- **Population:** População da Cidade.

In [83]:
# Leitura das Bases de Dados

df_raw_cases = pd.read_csv('./data/brazil_covid19_cities.csv')
df_raw_population = pd.read_excel('./data/brazil_population_2019.xlsx')
df_raw_cities = pd.read_excel('./data/brazil_cities_coordinates.xlsx')
df_raw_cities['city_code'] = df_raw_cities['city_code'].astype(str).str[:-1].astype(int)
# Bases Raw separadas em Base de Casos e Bases Geográficas

df_cases = df_raw_cases.copy()
df_cases['code'] = df_cases['code'].astype(int)
df_cases['cases'] = df_cases['cases'].astype(int)

df_geo = df_raw_population.merge(df_raw_cities, on = ["state_code", "city_code"], how = 'left').drop([ 'Unnamed: 8','city_name', 'health_region_code', 'health_region'], axis = 1)



# <font color='red' style='font-size: 40px;'> 2) Análise Exploratória </font>
<hr style='border: 2px solid red;'>

> 1. Para a análise, vamos explorar os seguintes tópicos:

- Distribuição Acumulada de Casos, Mortes, Taxa de Casos/População e Taxa de Mortes/População no Total, por Estado e as Top Cidades
- Distribuição ao longo do Tempo de Casos, Mortes, Taxa de Casos/População e Taxa de Mortes/População no Total, por Estado e as Top Cidades
- Mapa do Brasil colorido de acordo com a Taxa de Casos/População e Mortes/População

# <font color='green' style='font-size: 30px;'> 1.1) Distribuição Acumulada </font>
<hr style='border: 2px solid green;'>

# <font color='green' style='font-size: 30px;'> 1.2) Distribuição ao Longo do Tempo </font>
<hr style='border: 2px solid green;'>

# <font color='green' style='font-size: 30px;'> 1.3) Mapa do Brasil com a Taxa de Casos/População e Taxa de Mortes/População </font>
<hr style='border: 2px solid green;'>

In [ ]:
# df_raw_groupped = df_raw.groupby(['Date', 'Country/Region']).agg({
#     "Lat": "max", 
#     "Long": "max", 
#     "Confirmed": "sum", 
#     "Recovered": "sum", 
#     "Deaths": "sum"
# }).reset_index()

In [ ]:
# plt.figure(figsize=(16,8))
# ax = plt.subplot(1,2,1)
# ax.set_title('Casos Acumulados', fontsize=18, loc='left')
# plt.plot(df_brazil['Brazil'], label='Brazil')
# plt.plot(df_conf_w['Argentina'], label='Argentina')
# plt.plot(df_conf_w['Uruguay'], label='Uruguay')
# plt.xlabel("Dia")
# plt.ylabel("Casos")
# plt.legend();
# ax = plt.subplot(1,2,2)
# ax.set_title('Óbitos Acumulados', fontsize=18, loc='left')
# plt.plot(df_brazil['Brazil'], label='Brazil')
# plt.plot(df_deaths_w['Argentina'], label='Argentina')
# plt.plot(df_deaths_w['Uruguay'], label='Uruguay')
# plt.xlabel("Dia")
# plt.ylabel("Óbitos")
# plt.legend()